In [23]:
import requests
from bs4 import BeautifulSoup
import json
import string
import time
from requests.adapters import HTTPAdapter
import io
import glob

In [5]:
page = requests.get("https://www.set.or.th/set/companyprofile.do?symbol=ADVANC&ssoPageId=4&language=th&country=TH")


In [6]:
soup = BeautifulSoup(page.content, 'html.parser')
stock_code, thai_name = soup.findAll("div", {"class": "col-xs-12 col-md-12 col-lg-8"})[0].h3.text.split(" : ")
soup.findAll("div", {"class": "col-xs-9 col-md-5"})

[<div class="col-xs-9 col-md-5">18.77                     </div>,
 <div class="col-xs-9 col-md-5">3.59                     </div>,
 <div class="col-xs-9 col-md-5">10.51                     </div>,
 <div class="col-xs-9 col-md-5">585,699.78                     </div>,
 <div class="col-xs-9 col-md-5">SET</div>,
 <div class="col-xs-9 col-md-5">เทคโนโลยี</div>,
 <div class="col-xs-9 col-md-5">เทคโนโลยีสารสนเทศและการสื่อสาร</div>,
 <div class="col-xs-9 col-md-5">05 พ.ย. 2534</div>,
 <div class="col-xs-9 col-md-5">1.00 บาท                         </div>,
 <div class="col-xs-9 col-md-5">4,997,459,800.00 บาท</div>,
 <div class="col-xs-9 col-md-5">2,973,095,330.00 บาท</div>,
 <div class="col-xs-9 col-md-5"> - </div>,
 <div class="col-xs-9 col-md-5"> - </div>]

In [63]:
business_category = soup.findAll("div", {"class": "col-xs-9 col-md-5"})[6].text.strip()
industry_group = soup.findAll("div", {"class": "col-xs-9 col-md-5"})[5].text.strip()
par_price = float(soup.findAll("div", {"class": "col-xs-9 col-md-5"})[8].text.strip().split(" ")[0])
registered_fund = float(soup.findAll("div", {"class": "col-xs-9 col-md-5"})[9].text.strip().split(" ")[0].replace(",", ""))
net_registered_fund = float(soup.findAll("div", {"class": "col-xs-9 col-md-5"})[10].text.strip(" ").split(" ")[0].replace(",", ""))
business_category, industry_group, par_price, registered_fund, net_registered_fund

('เทคโนโลยีสารสนเทศและการสื่อสาร',
 'เทคโนโลยี',
 1.0,
 4997459800.0,
 2973095330.0)

In [27]:
high_stake_owner_page = requests.get("https://www.set.or.th/set/companyholder.do?symbol=ADVANC")

# How to hold info?

list of dictionary
where the key in the dictionary
ranking, name, stock_amt, percentage


In [49]:
stock_holder = []
holders = stake_soup.findAll("tbody")[0].findAll('tr')
for holder in holders:
    tds = holder.findAll('td')
    info = dict()
    info['ranking'] = int(tds[0].text.strip()[:-1])
    info['name'] = tds[1].text.strip()
    info['stock_amt'] = int(tds[2].text.replace(',', ''))
    info['percentage'] = float(tds[3].text)
    stock_holder.append(info)

stock_holder

[{'ranking': 1,
  'name': 'บริษัท อินทัช โฮลดิ้งส์ จำกัด (มหาชน)',
  'stock_amt': 1202712000,
  'percentage': 40.45},
 {'ranking': 2,
  'name': 'SINGTEL STRATEGIC INVESTMENTS PTE LTD.',
  'stock_amt': 693359000,
  'percentage': 23.32},
 {'ranking': 3,
  'name': 'บริษัท ไทยเอ็นวีดีอาร์ จำกัด',
  'stock_amt': 172311185,
  'percentage': 5.8},
 {'ranking': 4,
  'name': 'SOUTH EAST ASIA UK (TYPE C) NOMINEES LIMITED',
  'stock_amt': 123721953,
  'percentage': 4.16},
 {'ranking': 5,
  'name': 'สำนักงานประกันสังคม',
  'stock_amt': 81587800,
  'percentage': 2.74},
 {'ranking': 6,
  'name': 'STATE STREET EUROPE LIMITED',
  'stock_amt': 52949864,
  'percentage': 1.78},
 {'ranking': 7,
  'name': 'STATE STREET BANK AND TRUST COMPANY',
  'stock_amt': 18601776,
  'percentage': 0.63}]

# How to keep the data?


a stock will have the following dictionary keys
stock_code, thai_name, business_category, industry_group, par_price, registered_fund, net_registered_fund, stock_holders



In [3]:
def stock_info(stock_code, s):
    stock_prefix = stock_code.replace(" ", "+").replace("&", "%26")
    stock_page = s.get("https://www.set.or.th/set/companyprofile.do?symbol={}".format(stock_prefix), timeout=10)
    stake_page = s.get("https://www.set.or.th/set/companyholder.do?symbol={}".format(stock_prefix), timeout=10)
    stock_soup = BeautifulSoup(stock_page.content, 'html.parser')
    stake_soup = BeautifulSoup(stake_page.content, 'html.parser')
    stock_code, thai_name = stock_soup.findAll("div", {"class": "col-xs-12 col-md-12 col-lg-8"})[0].h3.text.split(" : ")
    market = stock_soup.findAll("div", {"class": "col-xs-9 col-md-5"})[4].text.strip()
    business_category = stock_soup.findAll("div", {"class": "col-xs-9 col-md-5"})[6].text.strip()
    industry_group = stock_soup.findAll("div", {"class": "col-xs-9 col-md-5"})[5].text.strip()
    par_price = float(stock_soup.findAll("div", {"class": "col-xs-9 col-md-5"})[8].text.strip().split(" ")[0])
    registered_fund = float(stock_soup.findAll("div", {"class": "col-xs-9 col-md-5"})[9].text.strip().split(" ")[0].replace(",", ""))
    net_registered_fund = float(stock_soup.findAll("div", {"class": "col-xs-9 col-md-5"})[10].text.strip(" ").split(" ")[0].replace(",", ""))
    info = dict(
        stock_code=stock_code,
        market=market,
        thai_name=thai_name,
        business_category=business_category,
        industry_group=industry_group,
        par_price=par_price,
        registered_fund=registered_fund,
        net_registered_fund=net_registered_fund
    )
    stock_holders = []
    holders = stake_soup.findAll("tbody")[0].findAll('tr')
    for holder in holders:
        tds = holder.findAll('td')
        holder_info = dict()
        holder_info['ranking'] = int(tds[0].text.strip()[:-1])
        holder_info['name'] = tds[1].text.strip()
        holder_info['stock_amt'] = int(tds[2].text.replace(',', ''))
        holder_info['percentage'] = float(tds[3].text)
        stock_holders.append(holder_info)
    info['stake_holders'] = stock_holders
    return info
    

In [4]:
def stock_list(prefix, s):
    list_page = s.get('https://www.set.or.th/set/commonslookup.do?&prefix={}'.format(prefix))
    list_soup = BeautifulSoup(list_page.content, 'html.parser')
    stock_list = []
    for stock in list_soup.findAll('table')[0].findAll('tr')[1:]:
        tds = stock.findAll('td')
        stock_list.append(tds[0].text.strip())
    return stock_list



In [9]:
upper_letters = ['NUMBER'] + list(string.ascii_uppercase)
upper_letters[3:]
forbidden_name = {'CON', 'PRN', 'AUX', 'NUL', 'COM1', 'COM2', 'COM3', 'COM4', 'COM5', 'COM6', 'COM7', 'COM8', 'COM9', 'COM0',
                 'LPT0', 'LPT1', 'LPT2', 'LPT3', 'LPT4', 'LPT5', 'LPT6', 'LPT7', 'LPT8', 'LPT9'}

upper_letters[20:]


['T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [10]:
s = requests.Session()
s.mount('https://www.set.or.th', HTTPAdapter(max_retries=5))
for letter in upper_letters[20:]:
    if letter == 'T':
        st_lst = stock_list(letter,s)
        st_lst = st_lst[st_lst.index('TPRIME'):]
    else:
        st_lst = stock_list(letter,s)
    for stock_code in st_lst:
        if stock_code in forbidden_name:
            stock_code += "_"
            print('forbidden windows name renaming with underscore at the end')
        with io.open("stock_data/{}.json".format(stock_code), 'w', encoding='utf8') as f:
            if stock_code[:-1] in forbidden_name:
                stock_code = stock_code[:-1]
            json.dump(stock_info(stock_code, s), f, ensure_ascii=False)
            stock_info(stock_code, s)
            time.sleep(1)
            print('done {}'.format(stock_code))
    

done TPRIME
done TR
done TRC
done TREIT
done TRITN
done TRT
done TRU
done TRUBB
done TRUE
done TSC
done TSE
done TSF
done TSI
done TSR
done TSTE
done TSTH
done TTA
done TTCL
done TTI
done TTL
done TTLPF
done TTTM
done TTW
done TU
done TU-PF
done TVD
done TVI
done TVO
done TVT
done TWP
done TWPC
done TWZ
done TYCN
done U
done UAC
done UBIS
done UEC
done UKEM
done UMI
done UMS
done UNIQ
done UOBKH
done UP
done UPA
done UPF
done UPOIC
done URBNPF
done UREKA
done UT
done UTP
done UV
done UVAN
done UWC
done VARO
done VCOM
done VGI
done VI
done VIBHA
done VIH
done VNG
done VNT
done VPO
done WACOAL
done WAVE
done WG
done WHA
done WHABT
done WHART
done WHAUP
done WICE
done WIIK
done WIN
done WINNER
done WORK
done WORLD
done WP
done WPH
done WR
done XO
done YCI
done YNP
done YUASA
done ZIGA
done ZMICO


# Data Merge

for ease of data analysis merge all the data into into a single json

In [22]:
datas = {}
for file in glob.iglob("stock_data/*.json"):
    with open(file, 'r', encoding='utf-8') as f:
        try:
            info = json.load(f)
            datas[info['stock_code']] = info
        except:
            pass
with io.open('all_stock.json', 'w', encoding='utf8') as f:
    json.dump(datas, f, ensure_ascii=False)
        